# Scalar quantisation in the transform domain and rate allocation via a greedy heuristic
This tutorial aims to show the effectiveness of frequency transformation in image compression. In particular we will consider a simple image codec which mimics the JPEG baseline compression standard as fas as the data partition and transform go. Then a scalar uniform quantiser is applied and a different quantisation step is derived on the basis of a simple rate allocation algorithm driven by a greedy heuristic. As usual, we will design and apply our simple image codec over the grey scale image `barbara.png`.

## Design of a simple transform-based image codec
To mimic the workflow of the JPEG compression standard in the Baseline profile, the incoming image will be partitioned according to a grid of non overlapping $8\times8$ blocks where for each one the Type-II Discrete Cosine Transform (DCT) will be applied, using floating point precision. The $8\times8$ blocks of transform coefficients will be then arranged to resemble a 3D array with size $\left(8,8,\frac{M\times N}{64}\right)$, being $M$ and $N$ the rows and columns of the input image. The following figure depicts the 3D array and its relation with the blocks of transform coefficients.

<img src="3D-array.png" width="650">

A total bit budget $B$ is available for quantisation and is spread across each 1D array with length $\frac{M\times N}{64}$, taken by considering position ($i$, $j$) for all $\frac{M\times N}{64}$ image blocks. Note that, given the definition of the 3D array above, there will be 64 of such 1D arrays. The whole bit budget $B$ is allocated using a greedy algorithm defined as follows.
 1. For each position ($i$, $j$), set its demand $d_{j\times8 + i}$ equal to the variance of the associated 1D array of transform coefficients.
 1. Set `total_bits = 0` and initialise all elements of the $8\times8$ matrix `bit_budget` to zero.
 1. Find the maximum of all demands $d_k$ and set $i= k$ \% $8$, where \% denotes the remainder operation and $j = k$ `>>` 3. Where `>>` denotes the binary right shift operator.
 1. Set `bit_budget[i,j] += 1` and `total_bits += 1`.
 1. Quantise all elements of the 1D array associated with ($i$, $j$) with a uniform quantiser having `bit_budget[i,j]` bits.
 1. Set $d_k = \sigma^2_e$, where $e$ denotes the reconstruction error
 1. If `total_bits <`$B$ goto 3 else terminate

The rate allocation strategy above is *greedy* because it assumes that the transform will do a good job in decorrelating the coefficients thus the number of bits should be allocated primarily to those coefficients showing a higher mean square error. This might not be optimal in case the transform didn't properly decorrelate all coefficients. However, we shall see that in practice the heuristic works quite well. To show that the transform operation is key in the rate allocation module (hence it is an effective coding tool), we will also apply the same rate allocation over a 3D which will contain image pixels only. In our analysis we will consider $B = 256$ bits which should correspond to an average of 4 bits per sample (being transform coefficient or image pixel).

## Implementation of the simple image codec and rate allocation
We will now implement the forward path of the encoding algorithm which comprises the following modules:
 * `make_partitioning`: To split up the input image into $8\times8$ blocks and create the aforementioned 3D array of samples.
 * `apply_dct`: To apply the Type-II DCT over each $8\times8$ block (i.e. plane) of the 3D array.
 * `rate_allocation`: To apply the rate allocation algorithm above given the input 3D array of samples (being either transform coefficients or image pixels).

Each module is implemented as Python method in the following code cell, so that it can be easily re-used.

In [ ]:
import math
from typing import Any

import cv2
import numpy as np
from nptyping import NDArray


def make_partitioning(image: NDArray[(Any, Any), np.int32], block_size: int) -> NDArray[(Any, Any, Any), np.int32]:
    rows, cols = image.shape[0], image.shape[1]
    rows_q = block_size * ((image.shape[0] + block_size - 1) // block_size)
    cols_q = block_size * ((image.shape[1] + block_size - 1) // block_size)
    z = rows_q * cols_q // block_size // block_size
    block_3d_array = np.zeros((block_size, block_size, z), np.int32)

    k = 0
    for r in range(0, rows_q, block_size):
        row_sel = slice(r, min(r + block_size, rows))
        for c in range(0, cols_q, block_size):
            col_sel = slice(c, min(c + block_size, cols))
            b = image[row_sel, col_sel]
            block_3d_array[:b.shape[0], :b.shape[1], k] = b
            k += 1

    return block_3d_array


def apply_dct(samples: NDArray[(Any, Any, Any), np.int32]) -> NDArray[(Any, Any, Any), np.float64]:
    coefficients = np.zeros(samples.shape)
    block_size = samples.shape[0]

    # Compute the Type-II DCT kernel matrix
    m1, m2 = np.meshgrid(range(block_size), range(block_size))
    normaliser = np.ones((block_size, block_size), np.float64)
    normaliser[0, ::] = 1.0 / math.sqrt(block_size)
    normaliser[1::, ::] = math.sqrt(2.0 / block_size)
    cosine_basis = np.cos(np.multiply(m2, 2.0 * m1 + 1.0) * np.pi / (2.0 * block_size))
    T = np.multiply(cosine_basis, normaliser)
    Tt = np.transpose(T)

    for z in range(samples.shape[2]):
        block = samples[:, :, z]
        coefficients[:, :, z] = np.matmul(T, np.matmul(block, Tt))

    return coefficients


def rate_allocation(samples: NDArray[(Any, Any, Any), np.float64], bit_budget: int) -> NDArray[(Any, Any), np.int32]:
    demand = np.var(samples, axis=2)
    block_y, block_x = samples.shape[0], samples.shape[1]  # we assume square blocks with size equal to a power of 2
    bit_allocation = np.zeros((block_y, block_x), np.int32)
    total_bits = 0

    while total_bits < bit_budget:
        k = np.argmax(demand)
        j, i = k // block_x, k % block_x
        bit_allocation[j, i] += 1
        dynamic_range = np.max(samples[j, i, :]) - np.min(samples[j, i, :])
        delta = dynamic_range / 2**bit_allocation[j, i]
        samples_rec = delta * np.round(samples[j, i, :] / delta)
        mse = np.average(np.square(samples_rec - samples[j, i, :]))
        demand[j, i] = mse
        total_bits += 1

    return bit_allocation

We note in the method `rate_allocation` that the quantisation step is dependent on the `dynamic_range`, which is equal to 256 when the method is applied over the image pixels whilst it depends on the value of the DCT subband in case of transform coefficients. One could assume that the DCT will add $\log_2(b)$ bits to the input pixels' bit depth, where $b$ denotes the block size used (8 in our example). However, depending on the image statistics, one could end up applying a coarser quantisation to some subbands. Accordingly, the `rate_allocation` method is designed assuming that the quantisation step will use a content dependent dynamic range and that such a quantisation step will be transmitted to the decoder on a per subband basis.

The following Python code cell implements the actual quantisation and inverse reconstruction path by implementing the following methods:
 * `reconstruct_samples`: To apply uniform quantisation and then samples' reconstruction (where here samples may be either transform coefficients or image pixels).
 * `apply_idct`: To compute the inverse DCT over the reconstructed transform coefficients.
 * `undo_partitioning`: To pick each $8\times8$ block from the 3D array of reconstructed samples and put it back to the final $M\times N$ image.

In [ ]:
def reconstruct_samples(samples: NDArray[(Any, Any, Any), np.float64], bit_budget: NDArray[(Any, Any), np.int32]) -> NDArray[(Any, Any, Any), np.float64]:
    reconstructed = np.zeros(samples.shape)

    for j in range(samples.shape[0]):
        for i in range(samples.shape[1]):
            dynamic_range = np.max(samples[j, i, :]) - np.min(samples[j, i, :])
            delta = dynamic_range / 2**bit_budget[j, i]
            reconstructed[j, i] = delta * np.round(samples[j, i, :] / delta)

    return reconstructed


def apply_idct(coefficients: NDArray[(Any, Any, Any), np.float64]) -> NDArray[(Any, Any, Any), np.int32]:
    samples = np.zeros(coefficients.shape, np.int32)
    block_size = coefficients.shape[0]

    # Compute the Type-II DCT kernel matrix
    m1, m2 = np.meshgrid(range(block_size), range(block_size))
    normaliser = np.ones((block_size, block_size), np.float64)
    normaliser[0, ::] = 1.0 / math.sqrt(block_size)
    normaliser[1::, ::] = math.sqrt(2.0 / block_size)
    cosine_basis = np.cos(np.multiply(m2, 2.0 * m1 + 1.0) * np.pi / (2.0 * block_size))
    T = np.multiply(cosine_basis, normaliser)
    Tt = np.transpose(T)

    for z in range(coefficients.shape[2]):
        block = coefficients[:, :, z]
        temp = np.round(np.matmul(Tt, np.matmul(block, T)))
        samples[:, :, z] = np.clip(temp, 0, 255).astype(np.int32)

    return samples


def undo_partitioning(samples: NDArray[(Any, Any, Any), np.int32], rows, cols) -> NDArray[(Any, Any), np.int32]:
    image = np.zeros((rows, cols), np.int32)
    block_size_y, block_size_x = samples.shape[0], samples.shape[1]
    rows_q, cols_q = block_size_y * ((rows + block_size_y - 1) // block_size_y), block_size_x * ((cols + block_size_x - 1) // block_size_x)

    z = 0
    for r in range(0, rows_q, block_size_y):
        row_sel = slice(r, r + block_size_y)
        for c in range(0, cols_q, block_size_x):
            col_sel = slice(c, c + block_size_x)
            image[row_sel, col_sel] = samples[:, :, z]
            z += 1

    return image[:rows, :cols]

We now have all methods to put together a simple image codec with the heuristic rate allocation. Accordingly, the following code cell will implement the full forward and backward processing chain over the input grey scale image `barbara.png`. Two rate allocations and associated processing will be investigated: one over the transform coefficients and the other over the simple images pixels.

In [ ]:
image = cv2.imread('../../input-data/barbara.png', cv2.IMREAD_UNCHANGED).astype(np.int32)
rows, cols = image.shape[0], image.shape[1]
block_size, total_bits = 8, 256

block_3d = make_partitioning(image, block_size)

coefficients = apply_dct(block_3d)

bit_budget_coefficients = rate_allocation(coefficients, total_bits)
bit_budget_pixels = rate_allocation(block_3d, total_bits)

coefficients_rec = reconstruct_samples(coefficients, bit_budget_coefficients)
block_3d_rec_pixels = reconstruct_samples(block_3d, bit_budget_pixels)

block_3d_rec_dct = apply_idct(coefficients_rec)
image_rec_dct = undo_partitioning(block_3d_rec_dct, rows, cols)
image_rec_pixels = undo_partitioning(block_3d_rec_pixels, rows, cols)

mse_dct = np.average(np.square(image - image_rec_dct))
mse_pixels = np.average(np.square(image - image_rec_pixels))
psnr_dct = 10 * np.log10(255**2 / mse_dct)
psnr_pixels = 10 * np.log10(255**2 / mse_pixels)

print(f"PSNR for transform domaind rate allocation {psnr_dct:.2f} [dB], pixel domain rate allocation {psnr_pixels:.2f} [dB]")

We can see from the PSNR values obtained that the rate allocation in the transform domain provided a better quality fidelity which can also be visualised in the following image cell.

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(25, 25))

plt.subplot(1, 3, 1), plt.imshow(image.astype(np.uint8), cmap='gray'), plt.title('Original', fontsize=16)
plt.subplot(1, 3, 2), plt.imshow(image_rec_dct.astype(np.uint8), cmap='gray'), plt.title('Rate allocation transform domain', fontsize=16)
plt.subplot(1, 3, 3), plt.imshow(image_rec_pixels.astype(np.uint8), cmap='gray'), plt.title('Rate allocation pixel domain', fontsize=16);

Also from the subjective point of view we can observe that the image reconstructed from the rate allocation in the pixel domain, shows some banding artefacts in the areas of the table's legs and the background. The banding artefacts are even more pronounced over an image such as the usual `cameraman.tif`: the application of the above processing over this other image is left as exercise to the reader. We can also visualise the rate allocation 2D arrays provided as output from the `rate_allocation` module in the following cell:

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(1, 2, 1), plt.imshow(bit_budget_pixels.astype(np.uint8), cmap='jet'), plt.title('Pixel domain', fontsize=16)
plt.subplot(1, 2, 2), plt.imshow(bit_budget_coefficients.astype(np.uint8), cmap='jet'), plt.title('Transform domain', fontsize=16);

As expected, when quantisation is operated over image pixels, the heuristic rate allocation will provide a flat bit budget profile since each one of the 64 sample will have equal chance to contribute to the final reconstructed quality: accordingly the rate allocation splits equally all 256 bits available for quantisation over all 64 groups of pixels. Conversely, when transform coefficients are considered, the bit budget profile is fairly non uniform and this is down to the energy compaction property of the DCT which tends to pack the signal's energy in a few coefficients, usually located in the low spatial frequency subbands. In this case, the simple heuristic considered will keep giving priority to these subbands as highlighted in the heat map above.

## Closing remarks
In this tutorial we have consider the case of rate allocation driven by a simple heuristic based on the demand associated with a group of samples. We considered two approaches to produce these group of samples: 1) simple pixels at a given $i,j$ in an $8\times 8$ block and 2) DCT subband associated with the $i,j$ spatial frequency. In the former case we obtained a uniform bit budget profile which, resembled 4 bits per samples and introduced banding artefacts. In the latter case the bit budget profile was more uneven, highlighting the energy compaction properties of a frequency transform such as the DCT and eventually leading to a better quality of the reconstructed pixels.